In [1]:
import tensorflow as tf
from Preprocess import label_for_nn
from alexnet_v2_model import model

In [2]:
benign_path = '/home/user01/Desktop/DDSM/4_benign_Pleo_10_ROI_subpic'#input('良性子图路径')#
cancer_path = '/home/user01/Desktop/DDSM/4_cancer_Pleo_10_ROI_subpic'#input('恶性子图路径')#
saver_path = '/home/user01/Desktop/DDSM/model/AlexNet_v2'#input('模型保存路径')#
size = 128
tf.reset_default_graph()

In [3]:
def get_trainsets_and_testsets(benign_path,cancer_path):
    (train_x ,train_y),(test_x,test_y) = label_for_nn(benign_path,cancer_path,scale=0.2,kind='RGB')
    train_x = train_x.reshape(train_x.shape[0],size,size,3).astype('float32')/255.0
    test_x = test_x.reshape(test_x.shape[0],size,size,3).astype('float32')/255.0
    return  test_x ,test_y

In [4]:
def conv2d_P(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='SAME')

In [5]:
def conv2d(x,W,s):
    return tf.nn.conv2d(x,W,strides=[1,s,s,1],padding='VALID')

In [6]:
def max_pooling2x2(x):
    return tf.nn.max_pool(x,ksize=[1,3,3,1],strides=[1,2,2,1],padding='VALID')

In [7]:
def cnn_test():
    x = tf.placeholder(tf.float32, [None, 128, 128, 3])
    y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)
    y_pre = model(x,keep_prob)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pre), reduction_indices=[1]))
    y_pre_label = tf.argmax(y_pre,1)
    y_label = tf.argmax(y,1)
    a = tf.add(y_pre_label, 1)
    b = tf.add(y_label, 3)
    c = tf.multiply(a, b)
    tp = tf.reduce_sum(tf.cast(tf.equal(c, 8), tf.float32))
    fp = tf.reduce_sum(tf.cast(tf.equal(c, 6), tf.float32))
    tn = tf.reduce_sum(tf.cast(tf.equal(c, 3), tf.float32))
    fn = tf.reduce_sum(tf.cast(tf.equal(c, 4), tf.float32))
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fp)
    tf.summary.scalar('test_Sensitivity', tpr)
    tf.summary.scalar('test_Specificity', tnr)
    correct_l1 = tf.equal(y_pre_label, y_label)
    accuracy = tf.reduce_mean(tf.cast(correct_l1, tf.float32))
    tf.summary.scalar('test_loss', cross_entropy)
    tf.summary.scalar('test_accuracy', accuracy)
    merged_summary_op = tf.summary.merge_all()
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        writer = tf.summary.FileWriter(saver_path, sess.graph)
        saver.restore(sess,saver_path+'/AlexNet_v2')
        test_accuracy = 0
        test_sensitivity = 0
        test_specificity = 0
        test_loss = 0
        for n in range(0,100):
            test_x, test_y = get_trainsets_and_testsets(benign_path,cancer_path)
            acc,sen,spec,loss,summary = sess.run([accuracy,tpr,tnr,cross_entropy,merged_summary_op],feed_dict={x:test_x,y:test_y,keep_prob:1.0})
            test_accuracy += acc
            test_sensitivity += sen
            test_specificity +=spec
            test_loss += loss
            print("{0:3d} accuracy:{1:>10.9f} loss:{2:>10.9f} specificity:{3:>10.9f} sensitivity:{4:>10.9f}".format(n,acc,loss,sen,spec))
            writer.add_summary(summary=summary, global_step=n)
        print("accuracy:{0:<6.3f}% loss:{1:<6.3f}% specificity:{2:<6.3f}% sensitivity:{3:<6.3f}%".format(test_accuracy,test_loss,test_sensitivity,test_specificity))

In [8]:
cnn_test()

INFO:tensorflow:Restoring parameters from /home/user01/Desktop/DDSM/model/AlexNet_v2/AlexNet_v2
  0 accuracy:0.958333313 loss:0.212489977 specificity:0.899999976 sensitivity:1.000000000
  1 accuracy:0.944444418 loss:0.178643256 specificity:0.884615362 sensitivity:1.000000000
  2 accuracy:0.921568632 loss:0.201185420 specificity:0.884615362 sensitivity:0.959999979
  3 accuracy:0.816326559 loss:0.358930320 specificity:0.740740716 sensitivity:0.909090936
  4 accuracy:0.911111116 loss:0.275641859 specificity:0.826086938 sensitivity:1.000000000
  5 accuracy:0.921568632 loss:0.252814561 specificity:0.869565189 sensitivity:0.964285731
  6 accuracy:0.958333313 loss:0.184910938 specificity:0.899999976 sensitivity:1.000000000
  7 accuracy:0.925925910 loss:0.197748408 specificity:0.884615362 sensitivity:0.964285731
  8 accuracy:0.869565189 loss:0.353030652 specificity:0.769230783 sensitivity:1.000000000
  9 accuracy:0.939393938 loss:0.264268816 specificity:0.857142866 sensitivity:1.000000000
 10 

 90 accuracy:0.893617034 loss:0.233058989 specificity:0.791666687 sensitivity:1.000000000
 91 accuracy:0.913043499 loss:0.290818393 specificity:0.862068951 sensitivity:1.000000000
 92 accuracy:0.944444418 loss:0.190126047 specificity:0.928571403 sensitivity:0.961538434
 93 accuracy:0.964285731 loss:0.185487032 specificity:0.923076928 sensitivity:1.000000000
 94 accuracy:0.934782624 loss:0.229712442 specificity:0.869565189 sensitivity:1.000000000
 95 accuracy:0.932203412 loss:0.209715024 specificity:0.920000017 sensitivity:0.941176474
 96 accuracy:0.933333337 loss:0.189172566 specificity:0.879999995 sensitivity:0.971428573
 97 accuracy:0.961538434 loss:0.168790564 specificity:0.923076928 sensitivity:1.000000000
 98 accuracy:0.936170220 loss:0.159723639 specificity:0.875000000 sensitivity:1.000000000
 99 accuracy:0.937500000 loss:0.241928682 specificity:0.869565189 sensitivity:1.000000000
accuracy:92.630% loss:22.630% specificity:86.782% sensitivity:98.261%
